In [7]:
import pandas as pd
import numpy as np
df_bl = pd.read_json('./baseline_results.json', lines=False)
df_bl_eq = np.where(df_bl['tags'] == df_bl['predicted'])
tp_bl = len(df_bl_eq[0])
acc_bl = tp_bl/len(df_bl.index)

df = pd.read_json('./baseline_simplified_data_results.json', lines=False)
df_eq = np.where(df['tags'] == df['predicted'])
tp = len(df_eq[0])
acc = tp/len(df.index)

print('Accuracy Baseline: ', acc_bl)
print('Accuracy Simplified Title: ', acc)

Accuracy Baseline:  0.5275
Accuracy Simplified Title:  0.5275


In [12]:
import importlib

numbers = ['2','3','4','5','6','7','8','9']
number_words = ['two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten']
currency_words = ['euro', 'yen', 'Euro','Yen']
currency_signs = ['€', '¥']
selection_multi = ['multi']
selection_phrase = ['phrase']
selection_passage = ['passage']
#baseline = importlib.import_module('transformer-baseline-task-1')

def has_number_word(inputString):
    return any(word in number_words for word in inputString.split())

def has_number(inputString):
    return any(char.isdigit() for char in inputString)

def has_currency_word(inputString):
    return any(word in currency_words for word in inputString.split())

def has_currency_sign(inputString):
    return any(char in currency_signs for char in inputString)

### baseline analysis

In [13]:
baseline_results = pd.read_json('./baseline_results.json', lines=False)

mask_multi_act = baseline_results.tags.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_act = baseline_results.tags.apply(lambda x: not any(item for item in selection_multi if item in x))
mask_multi_pred = baseline_results.predicted.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_pred = baseline_results.predicted.apply(lambda x: not any(item for item in selection_multi if item in x))

true_positive = len(baseline_results[mask_multi_pred & mask_multi_act].index)
false_positive = len(baseline_results[mask_multi_pred & mask_multi_not_act].index)
true_negative = len(baseline_results[mask_multi_not_pred & mask_multi_not_act].index)
false_negative = len(baseline_results[mask_multi_not_pred & mask_multi_act].index)

print('TP: ', true_positive)
print('FP: ', false_positive)
print('TN: ', true_negative)
print('FN: ', false_negative)

accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
print('Accuracy: ', accuracy)


TP:  56
FP:  18
TN:  639
FN:  87
Accuracy:  0.86875


### multi predicted by numbers contained in post text analysis

In [14]:
results = pd.read_json('./exported.json', lines=False)

mask_multi_act = results.tags.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_act = results.tags.apply(lambda x: not any(item for item in selection_multi if item in x))
mask_multi_pred = results.spoilerType_y.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_pred = results.spoilerType_y.apply(lambda x: not any(item for item in selection_multi if item in x))

true_positive = len(results[mask_multi_pred & mask_multi_act].index)
false_positive = len(results[mask_multi_pred & mask_multi_not_act].index)
true_negative = len(results[mask_multi_not_pred & mask_multi_not_act].index)
false_negative = len(results[mask_multi_not_pred & mask_multi_act].index)

print('TP: ', true_positive)
print('FP: ', false_positive)
print('TN: ', true_negative)
print('FN: ', false_negative)

accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
print('Accuracy: ', accuracy)

TP:  82
FP:  87
TN:  570
FN:  61
Accuracy:  0.815


### detect multipart spoilers by applying multiple conditions

In [22]:
df['st_contains_currency_sign_or_word'] = df['postText'].apply(lambda p: has_currency_sign(p) or has_currency_word(p))
df['st_contains_numbers_or_number_words'] = df['postText'].apply(lambda p: has_number(p) or has_number_word(p))

conditions = [
    ((df['st_contains_numbers_or_number_words'] == True) & (df['st_contains_currency_sign_or_word'] == False))
]

df['spoilerType'] = np.select(conditions,['multi'],'')
mask_multi_pred = df.spoilerType.apply(lambda x: any(item for item in selection_multi if item in x))
index_array = df[mask_multi_pred].index


In [23]:
results2 = pd.read_json('./baseline_results.json', lines=False)
results2.loc[index_array,'predicted'] = '[multi]'

mask_multi_act = results2.tags.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_act = results2.tags.apply(lambda x: not any(item for item in selection_multi if item in x))
mask_multi_pred = results2.predicted.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_pred = results2.predicted.apply(lambda x: not any(item for item in selection_multi if item in x))

true_positive = len(results2[mask_multi_pred & mask_multi_act].index)
false_positive = len(results2[mask_multi_pred & mask_multi_not_act].index)
true_negative = len(results2[mask_multi_not_pred & mask_multi_not_act].index)
false_negative = len(results2[mask_multi_not_pred & mask_multi_act].index)

print('TP: ', true_positive)
print('FP: ', false_positive)
print('TN: ', true_negative)
print('FN: ', false_negative)

accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
print('Accuracy: ', accuracy)

TP:  86
FP:  93
TN:  564
FN:  57
Accuracy:  0.8125


### target paragraph analysis

In [17]:
df = pd.read_json('./validation.jsonl', lines=True)
mask_multi_act = df.tags.apply(lambda x: any(item for item in selection_multi if item in x))
mask_phrase_act = df.tags.apply(lambda x: any(item for item in selection_phrase if item in x))
mask_passage_act = df.tags.apply(lambda x: any(item for item in selection_passage if item in x))
df['postText'] = df['postText'].apply(lambda p: p[0])
df['targetParagraphsAmount'] = df['targetParagraphs'].apply(lambda p: len(p))
average_multi_target_amounts = df[mask_multi_act]['targetParagraphsAmount'].mean()
average_phrase_target_amounts = df[mask_phrase_act]['targetParagraphsAmount'].mean()
average_passage_target_amounts = df[mask_passage_act]['targetParagraphsAmount'].mean()
median_multi_target_amounts = df[mask_multi_act]['targetParagraphsAmount'].median()
median_phrase_target_amounts = df[mask_phrase_act]['targetParagraphsAmount'].median()
median_passage_target_amounts = df[mask_passage_act]['targetParagraphsAmount'].median()
stdev_multi_target_amounts = df[mask_multi_act]['targetParagraphsAmount'].std()
stdev_phrase_target_amounts = df[mask_phrase_act]['targetParagraphsAmount'].std()
stdev_passage_target_amounts = df[mask_passage_act]['targetParagraphsAmount'].std()


print('multi: mean / stdev / median: ', average_multi_target_amounts, ' / ', stdev_multi_target_amounts, ' / ', median_multi_target_amounts)
print('phrase: mean / stdev / median: ', average_phrase_target_amounts, ' / ', stdev_phrase_target_amounts, ' / ', median_phrase_target_amounts)
print('passage: mean / stdev / median: ', average_passage_target_amounts, ' / ', stdev_passage_target_amounts, ' / ', median_passage_target_amounts)

multi: mean / stdev / median:  23.503496503496503  /  20.191991902339467  /  19.0
phrase: mean / stdev / median:  11.292537313432836  /  8.99955610071025  /  9.0
passage: mean / stdev / median:  13.869565217391305  /  14.000725586352  /  10.0


In [19]:
results3 = pd.read_json('./ap2_exported.json', lines=False)

mask_multi_act = results3.tags.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_act = results3.tags.apply(lambda x: not any(item for item in selection_multi if item in x))
mask_multi_pred = results3.spoilerType_y.apply(lambda x: any(item for item in selection_multi if item in x))
mask_multi_not_pred = results3.spoilerType_y.apply(lambda x: not any(item for item in selection_multi if item in x))

true_positive = len(results3[mask_multi_pred & mask_multi_act].index)
false_positive = len(results3[mask_multi_pred & mask_multi_not_act].index)
true_negative = len(results3[mask_multi_not_pred & mask_multi_not_act].index)
false_negative = len(results3[mask_multi_not_pred & mask_multi_act].index)

print('TP: ', true_positive)
print('FP: ', false_positive)
print('TN: ', true_negative)
print('FN: ', false_negative)

accuracy = (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative)
print('Accuracy: ', accuracy)

TP:  68
FP:  24
TN:  633
FN:  75
Accuracy:  0.87625
